<a href="https://colab.research.google.com/github/weso500/IOT-Anomaly-Detection/blob/main/Single_Image_VAE_C%2B_IGBT_I_GenLatent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtained intially from https://keras.io/examples/generative/vae/ and https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/generative/ipynb/vae.ipynb

In [1]:
pip install --upgrade keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1


In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import tensorflow as tf
import keras
from keras import ops
from keras import layers

In [3]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = keras.random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = keras.random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon

Use this notebook as example for obtaining images: https://colab.research.google.com/drive/1EOFDisibXb9a4A3lZ_2z4Jbhlnx0OVy6 The below takes care of obtaining images

In [4]:
latent_dim = 3

encoder_inputs = keras.Input(shape=(256, 256, 3))
x = layers.Conv2D(32, 3, activation="relu", strides=2, padding="same")(encoder_inputs)
x = layers.Conv2D(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation="relu")(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 128, 128, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 64, 64, 64)     │         18,496 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 262144)         │              0 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │     33,554,560 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_mean (Dense)            │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ z_log_var (Dense)         │ (None, 3)              │            387 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sampling (Sampling)       │ (None, 3)              │              0 │ z_mean[0][0],          │
│                           │                        │                │ z_log_var[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 33,574,726 (128.08 MB)

 Trainable params: 33,574,726 (128.08 MB)

 Non-trainable params: 0 (0.00 B)

In [5]:
latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(64 * 64 * 64, activation="relu")(latent_inputs)
x = layers.Reshape((64, 64, 64))(x)
x = layers.Conv2DTranspose(64, 3, activation="relu", strides=2, padding="same")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu", strides=2, padding="same")(x)
decoder_outputs = layers.Conv2DTranspose(3, 3, activation="sigmoid", padding="same")(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 3)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 262144)              │       1,048,576 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 256, 256, 32)        │          18,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 256, 256, 3)         │             867 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,104,835 (4.21 MB)

 Trainable params: 1,104,835 (4.21 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.layers import Layer
class KLLossLayer(Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
        kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))

        return kl_loss

class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")
        self.kl_loss_layer = KLLossLayer()

    def call(self, inputs):  # Add the call method
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        return reconstructed

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(
                    keras.losses.binary_crossentropy(data, reconstruction),
                    axis=(1, 2),
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        kl_loss = self.kl_loss_layer([z_mean, z_log_var])
        return reconstructed, kl_loss, z_mean, z_log_var

    def predict(self, x):
        reconstructed, kl_loss, z_mean, z_log_var = super().predict(x)
        return reconstructed, kl_loss, z_mean, z_log_var


In [7]:
!unzip /content/Gold_Folder_Final.zip

Streaming output truncated to the last 5000 lines.
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/108Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/109Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/10Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/110Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/111Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/112Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/113Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/114Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGBT-I/115Normal.png  
  inflating: Gold Folder Final/Training Data/Normal/Spectrograms_Normal_B+IGB

In [8]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_C+_IGBT-I',
  image_size=(256, 256),
  batch_size=32,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds = train_ds.map(lambda x: normalization_layer(x))

Found 500 files.


In [9]:
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_test_ds = test_ds.map(lambda x: normalization_layer(x))

Found 190 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT

In [10]:
test_anomaly_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Test_Data/Anomaly/Spectrograms_Anomaly_C+_IGBT-I',
  image_size=(256, 256),
  shuffle=False,
  batch_size=1,
  label_mode=None)

file_paths = test_ds.file_paths
print(file_paths)

normalization_layer = layers.Rescaling(scale= 1./255)

normalized_anomaly_test_ds = test_anomaly_ds.map(lambda x: normalization_layer(x))

Found 182 files.
['/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/500Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/501Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/502Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/503Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/504Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/505Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/506Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/507Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/508Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT-I/509Normal.png', '/content/Gold_Folder_Final/Test_Data/Normal/Spectrograms_Normal_C+_IGBT

In [11]:
#(x_train, _), (x_test, _) = keras.datasets.mnist.load_data()
#mnist_digits = np.concatenate([x_train, x_test], axis=0)
#mnist_digits = np.expand_dims(mnist_digits, -1).astype("float32") / 255

vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(normalized_ds, epochs=15, batch_size=32)

Epoch 1/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 25s 663ms/step - kl_loss: 343.6197 - loss: 43849.9219 - reconstruction_loss: 43506.3008
Epoch 2/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 19s 84ms/step - kl_loss: 519.6804 - loss: 30575.8086 - reconstruction_loss: 30056.1270
Epoch 3/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - kl_loss: 185.1988 - loss: 28695.1543 - reconstruction_loss: 28509.9590
Epoch 4/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 84ms/step - kl_loss: 129.1578 - loss: 27516.9648 - reconstruction_loss: 27387.8066
Epoch 5/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 86ms/step - kl_loss: 123.7850 - loss: 25814.8926 - reconstruction_loss: 25691.1094
Epoch 6/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - kl_loss: 113.5356 - loss: 23963.3184 - reconstruction_loss: 23849.7852
Epoch 7/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - kl_loss: 149.9042 - loss: 21874.5703 - reconstruction_loss: 21724.6680
Epoch 8/15
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 85ms/step - kl_loss: 121.4992 - loss: 21160.3672 - reconstruction_loss: 21038.8652
Epoch

In [12]:
train_ds_test_ = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/Gold_Folder_Final/Training_Data/Normal/Spectrograms_Normal_C+_IGBT-I',
  image_size=(256, 256),
  batch_size=1,
  label_mode=None)


normalization_layer = layers.Rescaling(scale= 1./255)

normalized_ds_test1 = train_ds_test_.map(lambda x: normalization_layer(x))

Found 500 files.


In [13]:
pip install sewar

  Preparing metadata (setup.py) ... done
  Created wheel for sewar: filename=sewar-0.4.6-py3-none-any.whl size=11419 sha256=957dcdbf12ace319f8fb927d87bae08a84b35a2056d02e7d409c12c3615d3b43
  Stored in directory: /root/.cache/pip/wheels/3f/af/02/9c6556ba287b62a945d737def09b8b8c35c9e1d82b9dfae84c
Successfully built sewar


In [14]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_ds_test1.take(500):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list



1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
0.0009572585959291345
45.648094
[[ 3.8052943 -4.640042  -6.865    ]]
[[-3.8563995 -3.3818429 -3.8429086]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0.000808282134001012
43.51299
[[ 6.0920777 -5.1291976 -3.6137722]]
[[-4.2664404 -4.976837  -4.266317 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.0008158803164711935
44.807915
[[ 4.0303435 -4.7163844 -6.539515 ]]
[[-3.876288  -3.5460136 -3.8699033]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
0.0009269469361128883
45.26956
[[ 3.749955 -4.597284 -6.873243]]
[[-3.8466854 -3.3328495 -3.8424518]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.0008084727461503872
45.392334
[[ 4.0480933 -4.716629  -6.617594 ]]
[[-3.8768625 -3.5360098 -3.8749912]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.0009219928340667899
46.482883
[[ 3.6442258 -4.6071024 -7.103466 ]]
[[-3.810474  -3.2727149 -3.8359535]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
0.0008202044452013313
43.36415
[[ 6.08198   -5.1257925 -3.5822036]]
[[-4.297084  -5.0143332 -4.2864594]]
1

In [15]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df = pd.DataFrame(dict)

# saving the dataframe
df.to_csv('C+_IGBT-I_train.csv')

In [16]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.0019503220299176085
52.275005
[[ 8.225758  -4.9141726 -0.3134616]]
[[-4.908333  -6.0618353 -4.650127 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
0.0019435356472904508
51.594196
[[ 8.1472435 -4.9039054 -0.3433935]]
[[-4.9163175 -6.0594945 -4.6495633]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
0.0021073778000777065
50.93879
[[ 8.053591  -4.914392  -0.6668171]]
[[-4.8509226 -5.9327455 -4.6173472]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.0021449607210638597
50.84589
[[ 8.049283  -4.9023137 -0.6442922]]
[[-4.8526015 -5.9545593 -4.6256785]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.0018657496987225007
51.07555
[[ 8.042      -4.961196   -0.65881115]]
[[-4.831297  -5.9808145 -4.597153 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.0019220993928468757
52.454926
[[ 8.24861   -4.9124827 -0.3370396]]
[[-4.9069834 -6.053087  -4.6447816]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
0.0019296319386643355
52.433807
[[ 8.25482   -4.894776  -0.2062925]]
[[-4.9362264 -6.10408   -4.66

In [17]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df2 = pd.DataFrame(dict)

# saving the dataframe
df2.to_csv('C+_IGBT-I_testnorm.csv')

In [18]:
import cv2
from urllib.request import AbstractBasicAuthHandler
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sewar.full_ref import mse, rmse, psnr, uqi, ssim, ergas, scc, rase, sam, msssim, vifp

#mse_values_normal = [] # Create an empty list to store mse values
#mse_values_anomaly = []
mse_values = []
kldiv1 = []
zmean1 = []
zmean2 = []
zmean3 = []
zlogvar1 = []
zlogvar2 = []
zlogvar3 = []

for elem in normalized_anomaly_test_ds.take(190):
  for i in range(len(elem)):
    elem_np = elem.numpy()
    exp = vae.predict(elem)
    exp_np = tf.get_static_value(exp)
    #plt.imshow(np.squeeze(elem_np))
    #plt.show()
    #plt.imshow(np.squeeze(exp_np[0]))
    #plt.show()
    #x = cv2.PSNR(elem_np, exp_np[0])
    x = mse(elem_np, exp_np[0])
    print(x)
    print(exp_np[1])
    kldiv1.append(exp_np[1])
    print(exp_np[2])
    zmean1.append(exp_np[2][0][0])
    zmean2.append(exp_np[2][0][1])
    zmean3.append(exp_np[2][0][2])
    print(exp_np[3])
    zlogvar1.append(exp_np[3][0][0])
    zlogvar2.append(exp_np[3][0][1])
    zlogvar3.append(exp_np[3][0][2])
    mse_values.append(x) # Append each mse value to the list

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
0.05644267931485127
42.843697
[[ 6.1148567 -4.5751224 -3.2788365]]
[[-5.7572236 -7.972235  -5.8776383]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0.011563508515583088
30.298447
[[ 5.0992546 -4.051278  -2.9726071]]
[[-4.139788  -4.1967382 -3.958661 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
0.025753898671960557
24.641277
[[ 4.178622  -3.7179909 -3.1843503]]
[[-3.7014017 -3.591259  -3.4824893]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
0.0009766239656037266
42.473583
[[ 4.1203427 -4.618156  -6.1738806]]
[[-3.9280677 -3.6146846 -3.9165053]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
0.04496799660527371
18.527153
[[ 4.2882915 -3.1132836 -1.6907744]]
[[-3.2169876 -2.8877764 -2.855541 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
0.022761991795493722
28.300455
[[ 4.456881  -3.9291997 -3.4322784]]
[[-4.1310215 -4.3165545 -4.023088 ]]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
0.000910565238829914
45.452614
[[ 3.8033397 -4.6111584 -6.8634014]]
[[-3.843525  -3.3264613 -3.8215544]]


In [19]:
import pandas as pd

dict = {'MSE': mse_values, 'KL Loss': kldiv1, 'Zmean1': zmean1, 'Zmean2': zmean2, 'Zmean3': zmean3, 'Zlogvar1': zlogvar1,'Zlogvar2': zlogvar2, 'Zlogvar3': zlogvar3 }

df3 = pd.DataFrame(dict)

# saving the dataframe
df3.to_csv('C+_IGBT-I_testanom.csv')

In [ ]:
from google.colab import files
files.download('/content/B+FLUX_testanom.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/B+FLUX_testnorm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/B+_FLUX_train.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>